In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

In [2]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [21]:
text = " I hate u"

In [22]:
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [23]:
import torch
from scipy.special import softmax

In [27]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.7468, -0.5561, -2.0293]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [24]:
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [25]:
import numpy as np

In [26]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    s = scores[ranking[i]]
    print(f"{i+1}) {np.round(float(s), 4)}")

1) 0.9567
2) 0.0352
3) 0.0081


In [47]:
from transformers import pipeline

# Create a sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis")

# Your sentence
sentence = "I hate you."

# Tokenize the sentence into words
words = sentence.split()

# Get the sentiment scores for each word
word_sentiments = {word: sentiment_analysis(word)[0]['score'] for word in words}

# Print the sentiment scores for each word
for word, sentiment_score in word_sentiments.items():
    print(f'{word}: {sentiment_score}')


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


I: 0.9797317385673523
hate: 0.9996899366378784
you.: 0.9996235370635986


In [48]:
{word: sentiment_analysis(word)[0] for word in words}

{'I': {'label': 'POSITIVE', 'score': 0.9797317385673523},
 'hate': {'label': 'NEGATIVE', 'score': 0.9996899366378784},
 'you.': {'label': 'POSITIVE', 'score': 0.9996235370635986}}

In [39]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

In [40]:
sid = SentimentIntensityAnalyzer()

In [45]:
sentence='i hate u'
words = nltk.word_tokenize(sentence)

# Get the sentiment scores for each word
word_sentiments = {word: sid.polarity_scores(word) for word in words}

# Print the sentiment scores for each word
# for word, sentiment_score in word_sentiments.items():
#     print(f'{word}: {sentiment_score}')

In [46]:
word_sentiments

{'i': {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0},
 'hate': {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound': -0.5719},
 'u': {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}}

In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

sentence = "I hate this product. It's amazing!"


In [11]:
# Tokenize the sentence into words
words = sentence.split()

# Get the sentiment scores for each word
word_sentiments = [sid.polarity_scores(word)['compound'] for word in words]


In [12]:
word_sentiments

[0.0, -0.5719, 0.0, 0.0, 0.0, 0.6239]

In [1]:
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def get_sentiment(word, pos):
    # Get the sentiment scores for a given word and part of speech
    synsets = list(swn.senti_synsets(word, pos))
    if synsets:
        # Use the average sentiment scores of all the synsets
        sentiment = sum(s.pos_score() - s.neg_score() for s in synsets) / len(synsets)
        return sentiment
    else:
        return 0.0

def get_word_sentiments(sentence):
    # Tokenize the sentence and get the part of speech for each word
    words = word_tokenize(sentence)
    pos_tags = pos_tag(words)

    # Map Penn Treebank POS tags to WordNet POS tags
    pos_mapping = {'N': 'n', 'V': 'v', 'R': 'r', 'J': 'a'}
    word_sentiments = []

    for word, pos in pos_tags:
        # Convert Penn Treebank POS tags to WordNet POS tags
        word_pos = pos[0].upper()
        if word_pos in pos_mapping:
            word_pos = pos_mapping[word_pos]

        # Get the sentiment score for the word
        sentiment = get_sentiment(word, word_pos)
        word_sentiments.append((word, sentiment))

    return word_sentiments

# Example usage
sentence = "I love this product. It's amazing!"
word_sentiments = get_word_sentiments(sentence)
for word, sentiment in word_sentiments:
    print(f"{word}: {sentiment}")

KeyError: 'P'

In [13]:
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def get_sentiment(word, pos):
    # Get the sentiment scores for a given word and part of speech
    synsets = list(swn.senti_synsets(word, pos))
    if synsets:
        # Use the average sentiment scores of all the synsets
        sentiment = sum(s.pos_score() - s.neg_score() for s in synsets) / len(synsets)
        return sentiment
    else:
        return 0.0

def get_word_sentiments(sentence):
    # Tokenize the sentence and get the part of speech for each word
    words = word_tokenize(sentence)
    pos_tags = pos_tag(words)

    # Map Penn Treebank POS tags to WordNet POS tags
    pos_mapping = {'N': 'n', 'V': 'v', 'R': 'r', 'J': 'a'}
    word_sentiments = []

    for word, pos in pos_tags:
        # Convert Penn Treebank POS tags to WordNet POS tags
        word_pos = pos[0].upper()
        word_pos = pos_mapping.get(word_pos, 'n')  # Use 'n' as the default value if not found

        # Get the sentiment score for the word
        sentiment = get_sentiment(word, word_pos)
        word_sentiments.append((word, sentiment))

    return word_sentiments

# Example usage
sentence = "I hate this product. It's amazing!"
word_sentiments = get_word_sentiments(sentence)
for word, sentiment in word_sentiments:
    print(f"{word}: {sentiment}")

I: 0.0
hate: -0.75
this: 0.0
product: 0.0
.: 0.0
It: 0.0
's: 0.0
amazing: 0.5
!: 0.0
